In [28]:
%pip install yfinance matplotlib

In [29]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error , r2_score

In [30]:
# download data

data = yf.download('RELIANCE.NS', start='2018-01-01', end='2025-12-31')
#

/tmp/ipython-input-4249081785.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('RELIANCE.NS', start='2018-01-01', end='2025-12-31')
[*********************100%***********************]  1 of 1 completed


In [31]:
data.head()

Price,Close,High,Low,Open,Volume
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS
Date,,,,,
2018-01-01,401.864838,407.585239,400.870942,407.585239,9453202
2018-01-02,402.483246,406.193799,400.385024,403.300454,9499419
2018-01-03,404.095551,409.042944,403.322514,408.601196,13507800
2018-01-04,406.525024,407.187631,404.493061,405.575300,9008932
2018-01-05,407.828156,409.440487,406.502971,407.187664,7441284


In [32]:
data = data[['Open','High','Low','Close','Volume']].copy()
print(data.head())

Price             Open        High         Low       Close      Volume
Ticker     RELIANCE.NS RELIANCE.NS RELIANCE.NS RELIANCE.NS RELIANCE.NS
Date                                                                  
2018-01-01  407.585239  407.585239  400.870942  401.864838     9453202
2018-01-02  403.300454  406.193799  400.385024  402.483246     9499419
2018-01-03  408.601196  409.042944  403.322514  404.095551    13507800
2018-01-04  405.575300  407.187631  404.493061  406.525024     9008932
2018-01-05  407.187664  409.440487  406.502971  407.828156     7441284


In [33]:
data.shape

(1975, 5)

In [34]:
# set target close price
data['target'] = data['Close'].shift(-1)

In [35]:
# drop last column
data = data.dropna()

In [36]:
data.head()

Price,Open,High,Low,Close,Volume,target
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,
Date,,,,,,
2018-01-01,407.585239,407.585239,400.870942,401.864838,9453202,402.483246
2018-01-02,403.300454,406.193799,400.385024,402.483246,9499419,404.095551
2018-01-03,408.601196,409.042944,403.322514,404.095551,13507800,406.525024
2018-01-04,405.575300,407.187631,404.493061,406.525024,9008932,407.828156
2018-01-05,407.187664,409.440487,406.502971,407.828156,7441284,410.169342


In [37]:
# create feature
# lagged price (previous 1- 5 day close)
for log in range(1,6):
  data[f'Close_log_{log}'] = data['Close'].shift(log)

In [38]:
# simple moving average --

data['SMA_5'] = data['Close'].rolling(window=5).mean()
data['SMA_20'] = data['Close'].rolling(window=20).mean()

# price change & volatility
data['Return_1'] = data['Close'].pct_change(1)
data['Return_5'] = data['Close'].pct_change(5)


# volume moving average
data['Volume_5'] = data['Volume'].rolling(window=5).mean()

In [39]:
data.head()

Price,Open,High,Low,Close,Volume,target,Close_log_1,Close_log_2,Close_log_3,Close_log_4,Close_log_5,SMA_5,SMA_20,Return_1,Return_5,Volume_5
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,,,,,,,,,,,
Date,,,,,,,,,,,,,,,,
2018-01-01,407.585239,407.585239,400.870942,401.864838,9453202,402.483246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-02,403.300454,406.193799,400.385024,402.483246,9499419,404.095551,401.864838,NaN,NaN,NaN,NaN,NaN,NaN,0.001539,NaN,NaN
2018-01-03,408.601196,409.042944,403.322514,404.095551,13507800,406.525024,402.483246,401.864838,NaN,NaN,NaN,NaN,NaN,0.004006,NaN,NaN
2018-01-04,405.575300,407.187631,404.493061,406.525024,9008932,407.828156,404.095551,402.483246,401.864838,NaN,NaN,NaN,NaN,0.006012,NaN,NaN
2018-01-05,407.187664,409.440487,406.502971,407.828156,7441284,410.169342,406.525024,404.095551,402.483246,401.864838,NaN,404.559363,NaN,0.003206,NaN,9782127.4


In [40]:
# drop blank row
data = data.dropna()

In [41]:
data.shape

(1955, 16)

In [42]:
data

Price,Open,High,Low,Close,Volume,target,Close_log_1,Close_log_2,Close_log_3,Close_log_4,Close_log_5,SMA_5,SMA_20,Return_1,Return_5,Volume_5
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,,,,,,,,,,,
Date,,,,,,,,,,,,,,,,
2018-01-29,426.800572,431.703766,423.752627,426.049622,9897064,419.821167,426.668060,426.866730,434.332001,429.141663,411.384094,428.611615,414.718060,-0.001449,0.035649,21216798.6
2018-01-30,426.623837,426.623837,419.202732,419.821167,9182956,424.636047,426.049622,426.668060,426.866730,434.332001,429.141663,426.747516,415.615877,-0.014619,-0.021719,13913243.2
2018-01-31,419.644482,426.049576,415.911843,424.636047,12551686,416.927826,419.821167,426.049622,426.668060,426.866730,434.332001,424.808325,416.723517,0.011469,-0.022324,11918218.4
2018-02-01,425.497429,429.627585,413.725279,416.927826,16524809,399.479401,424.636047,419.821167,426.049622,426.668060,426.866730,422.820544,417.365131,-0.018153,-0.023283,12365283.2
2018-02-02,413.018452,416.971950,397.999615,399.479401,28965593,398.507660,416.927826,424.636047,419.821167,426.049622,426.668060,417.382812,417.012849,-0.041850,-0.063723,15424421.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-22,1573.500000,1577.500000,1565.300049,1575.400024,10184852,1570.699951,1565.099976,1544.400024,1544.400024,1542.300049,1556.199951,1554.320020,1550.325006,0.006581,0.012338,9910292.2
2025-12-23,1580.900024,1580.900024,1568.099976,1570.699951,7506564,1558.199951,1575.400024,1565.099976,1544.400024,1544.400024,1542.300049,1560.000000,1551.875006,-0.002983,0.018414,9065233.8
